<a href="https://colab.research.google.com/github/Zyqzyqzyqzyq/skeleton-sp21/blob/master/world_tour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# project-description
The World Tour project is a combination of optimal transport and
resources allocation problems. The goal of the project is to organize
the best route to minimize the cost in terms of flight and staging, and
to maximize the total revenue. This problem can be visualized to a graph
filled with nodes which represents each city and lines between source
city and target city could be the route.

# Question list

#### Which 15 cities does Lady Gaga have the highest level of popularity on each continent?
Criteria of city selection in each continent:  
* Country selecting: Google Trends (Keyword: “Chromatical Ball” in past 12 months);
* Cities are chosen based on the past data of the world tour for Lady gaga;
* Each countries should have at most 3 cities (exclude countries in North America);


####The reference concert for choosing the cities and venues:
* The Fame Ball Tour (2009)
* The Monster Ball Tour (2009–2011)
* The Born This Way Ball (2012–2013)
* ArtRave: The Artpop Ball (2014)
* Joanne World Tour (2017–2018)

```
# This is formatted as code
```


* The Chromatica Ball (2022)


#### What is the objective function? How does the objective function relate to an optimal transport problem?
* Ticket Revunue - Cost of tour - Staging cost



#### What are the constraints?
* maximum capacity of each venue
* permitted flight tour between cities(domestic and international) 

#### Which two continents should be connected?
* North America -> South America -> Africa -> Oceania -> Asia -> Europe -> North America
* North America -> South America -> Africa -> Europe -> Asia -> Oceania -> North America


#### How do we visualize the cities that we have chosen to make a clear pattern of the problem? What tools? 
* By using Google my map to plot and connect the cities. 
* By using Distance.to calculating the distance of each cities.
#### What are the factors that we consider the starting city and target city for each continent?
* Distance and Permitted flight


#### Additional factors may need to be considered
* Should the flight between two cities really exist or we evaluate the theoretical optimal route only?
* Ticket Pricing: Based on past data and predicting the rate of increase.




# Examples of data collected and computations performed
* All cities venue capacity distance between cities. (show a table as an example here) 
* Price and quantity sold of tickets in each city/Revenue from advertising and sponsorships
* Total cost of staging a concert in each city/Travel costs between cities (flight, hotel..)
* Net profit/Price of ticket


In [ ]:
!pip install ortools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 9.2 MB/s 
     |████████████████████████████████| 408 kB 60.4 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
tensorflow-metadata 1.10.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.9 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.9 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.9 which is incompatible.
google-api-core 1.31.

In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    data = {}
    data['distance_matrix'] = [
        [0, 852, 339,	362,	1676,	1728,	2587,	3458,	5134,	5523,	4333,	1124],
        [852,	0,	546	,1127,	844,	957,	1878,	3334,	5631,	5673,	4572,	822],
        [339,	546	,0,	680	,1345	,1389,	2252,	3275,	5217,	5453,	4295,	836],
        [362,	1127,	680,	0	,1970	,2050,	2930,	3783,	5238,	5765,	4554,	1486],
        [1676,	844,	1345,	1970,	0	,282,	1140,	3141,	5964,	5650,	4676,	1043],
        [1728,	957,	1385,	2050,	282	,0,	930	,2863,	5754,	5381,	4425,	909],
        [2587	,1878,	2252,	2930,	1140,	930	,0,	2469,	5799,	5034,	4263,	1561],
        [3458,	3334,	3275,	3783,	3141,	2863,	2469,	0	,3602,	2567,	1894,	2520],
        [5134,	5631,	5217,	5238,	5964,	5754,	5799,	3602,	0	,2158	,1759	,4955],
        [5523,	5673,	5453,	5765,	5650,	5381,	5034,	2567,	2158,	0	,1249	,4859],
        [4333,	4572,	4295,	4554,	4676,	4425,	4263,	1894	,1759	,1249,	0	,3784],
        [1124,	822,	836,	1486,	1043,	909	,1561,	2520,	4955,	4859,	3784,	0],
    ]  # distance in Km
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} kilometers'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}kilometers\n'.format(route_distance)


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)


main()



Objective: 17332 kilometers
Route for vehicle 0:
 0 -> 2 -> 1 -> 11 -> 4 -> 5 -> 6 -> 7 -> 10 -> 9 -> 8 -> 3 -> 0



## South America Cities Tour 

Cities:

0. Sao Paulo

1. Porto Alegre

2. Curitiba

3. Rio de Janeiro

4. Buenos Aires

5. Rosario

6. Santiago

7. Lima

8. San Juan

9. San José

10. Bogotá

11. Asunción

In [ ]:
 [0, 852, 339,	362,	1676,	1728,	2587,	3458,	5134,	5523,	4333,	1124],
        [852,	0,	546	,1127,	844,	957,	1878,	3334,	5631,	5673,	4572,	822],
        [339,	546	,0,	680	,1345	,1389,	2252,	3275,	5217,	5453,	4295,	836],
        [362,	1127,	680,	0	,1970	,2050,	2930,	3783,	5238,	5765,	4554,	1486],
        [1676,	844,	1345,	1970,	0	,282,	1140,	3141,	5964,	5650,	4676,	1043],
        [1728,	957,	1385,	2050,	282	,0,	930	,2863,	5754,	5381,	4425,	909],
        [2587	,1878,	2252,	2930,	1140,	930	,0,	2469,	5799,	5034,	4263,	1561],
        [3458,	3334,	3275,	3783,	3141,	2863,	2469,	0	,3602,	2567,	1894,	2520],
        [5134,	5631,	5217,	5238,	5964,	5754,	5799,	3602,	0	,2158	,1759	,4955],
        [5523,	5673,	5453,	5765,	5650,	5381,	5034,	2567,	2158,	0	,1249	,4859],
        [4333,	4572,	4295,	4554,	4676,	4425,	4263,	1894	,1759	,1249,	0	,3784],
        [1124,	822,	836,	1486,	1043,	909	,1561,	2520,	4955,	4859,	3784,	0],]

In [7]:
import json

import flask
import numpy as np
from flask import request, jsonify
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

app = flask.Flask(__name__)
app.config["DEBUG"] = True


def create_data_model(distance_input_matrix):
    """Stores the data for the problem."""
    data = {'distance_matrix': distance_input_matrix, 'num_vehicles': 1, 'depot': 0}
    data['distance_matrix'] = [
        [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
        [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
        [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
        [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
        [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
       [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
         [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
         [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
         [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
         [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
        [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
         [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
         [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
     ]  # yapf: disable
    return data


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    # print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    # plan_output = 'Route for vehicle 0:\n'
    plan_output = ''
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += '{}->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += '{}'.format(manager.IndexToNode(index))
    return plan_output
    # print(plan_output)
    # plan_output += 'Route distance: {}miles\n'.format(route_distance)


def main(distance_input_matrix=None):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(distance_input_matrix)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        return print_solution(manager, routing, solution)


@app.route('/', methods=['GET'])
def home():
    return main()


@app.route('/optimize', methods=['POST'])
def optimize():
    if request.json:
        data_matrix = request.json['data']
        narrows = len(data_matrix)  # 3 rows in your example
        narcs = len(data_matrix[0])
        a = np.zeros((narrows + 1, narcs + 1), dtype='int32').tolist()
        for i in range(len(data_matrix)):
            for j in range(len(data_matrix[i])):
                a[i][j] = data_matrix[i][j]
        #result = main(a)
        result = main(data_matrix)
        r_l = result.split('->')
        #r_l.remove(str(narrows))
        return jsonify({'data': r_l})


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
